In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
EMBEDDINGS_PATH = '/content/drive/MyDrive/DSN Project/CNN-GRU/muse_embeddings'
DATA_DIR = '/content/drive/MyDrive/DSN Project/CNN-GRU/data'

In [ ]:
lang_map = {
  'german':'de',
  'arabic':'ar',
  'indonesian':'id',
  'italian':'it',
  'portuguese':'pt',
  'spanish':'es',
  'turkish':'tr',
  'danish':'da',
  'french':'fr',
  'hindi':'hi',
  'english':'en',
}
CSV_FILE_PATH = '/content/drive/MyDrive/DSN Project/CNN-GRU/data'

# (ignore) Data preprocessing

In [ ]:
danish_df = pd.read_csv(f'{DATA_DIR}/B_danish_processed.csv')
danish_df.to_csv(f'{DATA_DIR}/danish_final.csv', header=False, index=False)

In [ ]:
english_df = pd.read_csv(f'{DATA_DIR}/B_english_processed.csv', index_col=0)
english_df.to_csv(f'{DATA_DIR}/english_final.csv', header=False, index=False)

In [ ]:
french_df = pd.read_csv(f'{DATA_DIR}/B_french_processed.csv')
french_df.to_csv(f'{DATA_DIR}/french_final.csv', header=False, index=False)

In [ ]:
german_df = pd.read_csv(f'{DATA_DIR}/B_german_processed.csv', index_col=0)
german_df.to_csv(f'{DATA_DIR}/german_final.csv', header=False, index=False)

In [ ]:
arabic_df = pd.read_csv(f'{DATA_DIR}/B_arabic_processed.csv')
arabic_df.to_csv(f'{DATA_DIR}/arabic_final.csv', header=False, index=False)

In [ ]:
hindi_df = pd.read_csv(f'{DATA_DIR}/B_hindi_processed.csv')
hindi_df.to_csv(f'{DATA_DIR}/hindi_final.csv', header=False, index=False)

In [ ]:
indonesian_df = pd.read_csv(f'{DATA_DIR}/B_indonesian_processed.csv', index_col=0)
indonesian_df.to_csv(f'{DATA_DIR}/indonesian_final.csv', header=False, index=False)

In [ ]:
italian_df = pd.read_csv(f'{DATA_DIR}/B_italian_processed.csv', index_col=0)
italian_df.to_csv(f'{DATA_DIR}/italian_final.csv', header=False, index=False)

In [ ]:
portuguese_df = pd.read_csv(f'{DATA_DIR}/B_portuguese_processed.csv')
portuguese_df.to_csv(f'{DATA_DIR}/portuguese_final.csv', header=False, index=False)

In [ ]:
spanish_df = pd.read_csv(f'{DATA_DIR}/B_spanish_processed.csv', index_col=0)
spanish_df.to_csv(f'{DATA_DIR}/spanish_final.csv', header=False, index=False)

In [ ]:
turkish_df = pd.read_csv(f'{DATA_DIR}/B_turkish_processed.csv')[['text', 'hs']]
turkish_df.to_csv(f'{DATA_DIR}/turkish_final.csv', header=False, index=False)

# Installation

In [ ]:
!pip install torch
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 61 kB 583 kB/s 
     |████████████████████████████████| 3.3 MB 31.4 MB/s 
     |████████████████████████████████| 596 kB 88.5 MB/s 
     |████████████████████████████████| 895 kB 71.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# !conda install cudatoolkit==10.1
!nvidia-smi

Tue Dec  7 05:51:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   3361      0 --:--:-- --:--:-- --:--:--  3361
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9 MB 189 kB/s 
     |████████████████████████████████| 1.0 MB 46.7 MB/s 
+ set +ex
Installation succeeded!


# Model Preparation

In [ ]:
# ML/data imports
import numpy as np
from transformers import AdamW
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score,roc_auc_score,classification_report,accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from itertools import chain, repeat, islice
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset

# utils imports
import time
import datetime
from collections import Counter
import collections
import torch
import io
import glob
import csv
import random

## Monolingual CNN-GRU model

In [ ]:
import torch
import torch.nn as nn
import numpy as np

def global_max_pooling(tensor, dim, topk):
    """Global max pooling"""
    ret, _ = torch.topk(tensor, topk, dim)
    return ret

class CNN_GRU(nn.Module):
    def __init__(self,args,vector):
        super(CNN_GRU, self).__init__()
        self.embedsize = vector.shape[1]
        self.conv1 = nn.Conv1d(self.embedsize,100, 2)
        self.conv2 = nn.Conv1d(self.embedsize,100, 3,padding=1)
        self.conv3 = nn.Conv1d(self.embedsize,100, 4,padding=2)
        self.maxpool1D = nn.MaxPool1d(4, stride=4)
        self.seq_model = nn.GRU(100, 100, bidirectional=False, batch_first=True)
        self.embedding = nn.Embedding(args["vocab_size"], self.embedsize)
        self.embedding.weight = nn.Parameter(torch.tensor(vector.astype(np.float32), dtype=torch.float32))
        self.embedding.weight.requires_grad = args["train_embed"]
        self.num_labels=2
        self.weights=args['weights']
        self.out = nn.Linear(100, self.num_labels)

        
    def forward(self,x,labels=None):
        batch_size=x.size(0)
        h_embedding = self.embedding(x).to(device)
 
        new_conv1=self.maxpool1D(self.conv1(h_embedding.permute(0,2,1))).to(device)
        new_conv2=self.maxpool1D(self.conv2(h_embedding.permute(0,2,1))).to(device)
        new_conv3=self.maxpool1D(self.conv3(h_embedding.permute(0,2,1))).to(device)
        concat=self.maxpool1D(torch.cat([new_conv1, new_conv2,new_conv3], dim=2).to(device)).to(device)
        h_seq, _ = self.seq_model(concat.permute(0,2,1))
        global_h_seq=torch.squeeze(global_max_pooling(h_seq, 1, 1)) 
        output=self.out(global_h_seq)
        
        if labels is not None:
        	loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(self.weights).to(device))
        	loss = loss_fct(output.view(-1, self.num_labels), labels.view(-1))
        	return loss,output
        return output


## Multilingual CNN-GRU model + dataset

In [ ]:
LANG_CODE_MAP = {k: i for i, k in enumerate(lang_map)}

In [ ]:
CODE_LANG_MAP = {v: k for k, v in LANG_CODE_MAP.items()}

In [ ]:
CODE_LANG_MAP

{0: 'german',
 1: 'arabic',
 2: 'indonesian',
 3: 'italian',
 4: 'portuguese',
 5: 'spanish',
 6: 'turkish',
 7: 'danish',
 8: 'french',
 9: 'hindi',
 10: 'english'}

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from collections import defaultdict


def global_max_pooling(tensor, dim, topk):
    """Global max pooling"""
    ret, _ = torch.topk(tensor, topk, dim)
    return ret

EMBEDDING_SIZE = 300
class MULTI_CNN_GRU(nn.Module):
    def __init__(self, args, lang_to_vector):
        super(MULTI_CNN_GRU, self).__init__()
        self.lang_to_embedding = {}
        self.embedsize = EMBEDDING_SIZE
        for lang in lang_to_vector:
          vector = lang_to_vector[lang]
          vocabsize = vector.shape[0]
          embedding = nn.Embedding(vocabsize, self.embedsize)
          embedding.weight = nn.Parameter(torch.tensor(vector.astype(np.float32), dtype=torch.float32))
          embedding.weight.requires_grad = args["train_embed"]
          self.lang_to_embedding[lang] = embedding

        self.conv1 = nn.Conv1d(self.embedsize,100, 2)
        self.conv2 = nn.Conv1d(self.embedsize,100, 3,padding=1)
        self.conv3 = nn.Conv1d(self.embedsize,100, 4,padding=2)
        self.maxpool1D = nn.MaxPool1d(4, stride=4)
        self.seq_model = nn.GRU(100, 100, bidirectional=False, batch_first=True)
        
        self.num_labels=2
        self.weights=args['weights']
        self.out = nn.Linear(100, self.num_labels)

        
    def forward(self, inputs, labels=None, langs=None):
        batch_size=inputs.size(0)
        lang_to_data = defaultdict(lambda:defaultdict(list))
        # Create dict lang -> (input, label)
        for i, lang in enumerate(langs):
            lang_to_data[lang]['inputs'].append(inputs[i].to(device))
            if labels is not None:
              lang_to_data[lang]['labels'].append(labels[i].to(device))
        # create embeddings from inputs of each language
        embeddings = []
        for lang in lang_to_data:
            # print(lang_to_data[lang]['inputs'])
            lang_x = lang_to_data[lang]['inputs'][0].to(device)
            embeddings.append(self.lang_to_embedding[CODE_LANG_MAP[lang.item()]](lang_x))

        # for i in range(len(embeddings)):
        #   embeddings[i] = torch.reshape(embeddings[i], (batch_size, embeddings[i].shape[0] // batch_size, embeddings[i].shape[1]))
        # concat embeddings for all languages
        # for embedding in embeddings:
        #     print(embedding.shape)
        # concatted_embedding = torch.cat(embeddings, dim=0).to(device)
        concatted_embedding = torch.stack(embeddings).to(device)
        new_conv1=self.maxpool1D(self.conv1(concatted_embedding.permute(0,2,1))).to(device)
        new_conv2=self.maxpool1D(self.conv2(concatted_embedding.permute(0,2,1))).to(device)
        new_conv3=self.maxpool1D(self.conv3(concatted_embedding.permute(0,2,1))).to(device)
        concat=self.maxpool1D(torch.cat([new_conv1, new_conv2,new_conv3], dim=2).to(device)).to(device)
        h_seq, _ = self.seq_model(concat.permute(0,2,1))
        global_h_seq=torch.squeeze(global_max_pooling(h_seq, 1, 1)) 
        output=self.out(global_h_seq)
        
        if labels is not None:
        	loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(self.weights).to(device))
        	loss = loss_fct(output.view(-1, self.num_labels), labels.view(-1))
        	return loss,output
        return output


In [ ]:
class MonolingualDataset(Dataset):
    """Monolingual Hatespeech dataset."""

    def __init__(self, data, lang, word2id): 
        """
        Args:
            lang_to_data (dict): lang -> data (input_vec, label)
        """
        self.dataset = []
        self.data_input = []
        self.data_labels = []
        self.data_langs = []
        self.word2id = word2id
        for datum in data:
            input_id, label, orig_text = datum
            self.dataset.append([input_id, label, lang])
            self.data_input.append(input_id)
            self.data_labels.append(label)
            self.data_langs.append(lang)


    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

        input = self.data_input[idx]
        label = self.data_labels[idx]
        language = self.data_langs[idx]

        MAX_VECTOR_LENGTH = 768
        if len(input) < MAX_VECTOR_LENGTH:
          input.extend([len(list(self.word2id.keys()))+1] * (MAX_VECTOR_LENGTH - len(input)))
        else:
          input = input[:MAX_VECTOR_LENGTH]

        # if len(input) != 427:
        #   input.extend([len(list(self.lang_to_word2id[language].keys()))+1] * (427 - len(input)))

        sample = (torch.tensor(input), torch.tensor([label], dtype=torch.long), language)

        return sample

In [ ]:
class MultilingualHatespeechDataset(Dataset):
    """Multilingual Hatespeech dataset."""

    def __init__(self, lang_to_data, lang_to_word2id): 
        """
        Args:
            lang_to_data (dict): lang -> data (input_vec, label)
        """
        self.dataset = []
        self.data_input = []
        self.data_labels = []
        self.data_langs = []
        self.lang_to_word2id = lang_to_word2id
        for lang in lang_to_data:
            input_data = lang_to_data[lang]
            for datum in input_data:
                input_id, label, orig_text = datum
                self.dataset.append([input_id, label, lang])
                self.data_input.append(input_id)
                self.data_labels.append(label)
                self.data_langs.append(lang)


    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

        input = self.data_input[idx]
        label = self.data_labels[idx]
        language = self.data_langs[idx]

        MAX_VECTOR_LENGTH = 768
        if len(input) < MAX_VECTOR_LENGTH:
          input.extend([len(list(self.lang_to_word2id[language].keys()))+1] * (MAX_VECTOR_LENGTH - len(input)))
        else:
          input = input[:MAX_VECTOR_LENGTH]

        # if len(input) != 427:
        #   input.extend([len(list(self.lang_to_word2id[language].keys()))+1] * (427 - len(input)))

        sample = (torch.tensor(input), torch.tensor([label], dtype=torch.long), language)

        return sample

## Eval Utils

In [ ]:
# Get accuracy to evaluate model 
# Used by: Eval_phase_cnngru
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# data_type = train/test/val
def Eval_phase_cnngru(params, data_type, model, data=None, data_loader=None):
    model.eval()
    print("Running eval on ",data_type,"...")
    t0 = time.time()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    true_labels=[]
    pred_labels=[]

    for i, batch in enumerate(data_loader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_labels = batch

      # Telling the model not to compute or store gradients, saving memory and
      # speeding up validation
      with torch.no_grad():        
        outputs = model(b_input_ids)

      logits = outputs
      # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()

      label_ids = b_labels.to('cpu').numpy()
      # Calculate the accuracy for this batch of test sentences.
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)
      # Accumulate the total accuracy.
      eval_accuracy += tmp_eval_accuracy

      pred_labels+=list(np.argmax(logits, axis=1).flatten())
      true_labels+=list(label_ids.flatten())

      # Track the number of batches
      nb_eval_steps += 1

    # Get the accuracy and macro f1 scores
    testf1=f1_score(true_labels, pred_labels, average='weighted')
    testauc=roc_auc_score(true_labels, pred_labels, average='weighted')
    testacc=accuracy_score(true_labels,pred_labels)

    # Report the final accuracy for this validation run.
    print(f" Accuracy: {testacc}")
    print(" Fscore: {0:.2f}".format(testf1))
    print(" Roc Auc score: {0:.2f}".format(testauc))
    print(" Test took: {:}".format(format_time(time.time() - t0)))
    
    return testf1,testacc,testauc

In [ ]:
# data_type = train/test/val
# TBD!!!
def Eval_phase_multi_cnngru(params, data_type, model, data_loader):
    model.eval()
    print("Running eval on ",data_type,"...")
    t0 = time.time()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    true_labels=[]
    pred_labels=[]

    for i, batch in enumerate(data_loader):
      # Add batch to GPU
      # Unpack the inputs from our dataloader
      b_input_ids, b_labels, b_langs = batch
      b_langs = torch.tensor([LANG_CODE_MAP[lang] for lang in b_langs])
      b_input_ids, b_labels, b_langs =  b_input_ids.to(device), b_labels.to(device), b_langs.to(device)
      # Telling the model not to compute or store gradients, saving memory and
      # speeding up validation
      with torch.no_grad():        
        outputs = model(b_input_ids, langs=b_langs)

      logits = outputs
      # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()

      label_ids = b_labels.to('cpu').numpy()
      # Calculate the accuracy for this batch of test sentences.
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)
      # Accumulate the total accuracy.
      eval_accuracy += tmp_eval_accuracy

      pred_labels+=list(np.argmax(logits, axis=1).flatten())
      true_labels+=list(label_ids.flatten())

      # Track the number of batches
      nb_eval_steps += 1

    # Get the accuracy and macro f1 scores
    testf1=f1_score(true_labels, pred_labels, average='weighted')
    testauc=roc_auc_score(true_labels, pred_labels, average='weighted')
    testacc=accuracy_score(true_labels,pred_labels)

    # Report the final accuracy for this validation run.
    print(f" Accuracy: {testacc}")
    print(" Fscore: {0:.2f}".format(testf1))
    print(" Roc Auc score: {0:.2f}".format(testauc))
    print(" Test took: {:}".format(format_time(time.time() - t0)))
    
    return testf1,testacc,testauc

## Data Utils

In [ ]:
def fix_the_random(seed_val = 42):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# TORCH DATA GEN
# def most_frequent(List): 
#     counter=Counter(List)
#     max_freq= max(set(List), key = List.count)
#     return max_freq,counter[max_freq]


# def CheckForGreater(list1, val):  
#     return(all(x > val for x in list1))  

def pad_infinite(iterable, padding=None):
       return chain(iterable, repeat(padding))

def pad(iterable, size, padding=None):
       return islice(pad_infinite(iterable, padding), size)

def encode_data(df,word2id):
    max_len=0
    for index,row in tqdm(df.iterrows(),total=len(df)):
        
        if(max_len<len(row["text"].split(' '))):
            max_len=len(row["text"].split(' '))

    tuple_new_data=[]


    for index,row in df.iterrows():
        list_token_id=[]
        words=row['text'].split(' ')
        for word in words:
            try:
                index=word2id[word]
            except KeyError:
                index=len(list(word2id.keys()))
            list_token_id.append(index)
        with_padding_text=list(pad(list_token_id, max_len, len(list(word2id.keys()))+1))
        tuple_new_data.append((with_padding_text,row['label'],row['text']))
    return tuple_new_data

In [ ]:
# Function to load the MUSE embeddings
def load_vec(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [ ]:
# Split csv into text and labels (Neha)
def split_csv_into_xy(file_path):
    f = open(file_path)
    csvreader = csv.reader(f)
    text = []
    labels = []
    for i, row in enumerate(csvreader):
        t, l = row
        text.append(t)
        labels.append(l)
    f.close()
    return text, labels

# Convert train,test,val splits to df
def create_data_dfs(X_train, y_train, X_test, y_test, X_val, y_val, params):  
    # TRAIN
    df_train = pd.concat([pd.DataFrame(X_train, columns=['text']), pd.DataFrame(y_train, columns=['label'])], axis=1)
    if(params['take_ratio']==True):
        n_samples=int(len(df_train)*params['sample_ratio']/100)
    else:
        n_samples=params['sample_ratio']

    if(n_samples==0): 
        n_samples+=1
    df_train=stratified_sample_df(df_train, 'label', n_samples,params['samp_strategy'],params['random_seed'])
    df_train['label'] = df_train['label'].astype('int32')

    #TEST
    df_test = pd.concat([pd.DataFrame(X_test, columns=['text']), pd.DataFrame(y_test, columns=['label'])], axis=1)
    df_test['label'] = df_test['label'].astype('int32')

    #VAL
    df_val = pd.concat([pd.DataFrame(X_val, columns=['text']), pd.DataFrame(y_val, columns=['label'])], axis=1)
    df_val['label'] = df_val['label'].astype('int32')

    return df_train, df_test, df_val

In [ ]:
def stratified_sample_df(df, col, n_samples,sampled='stratified',random_state=1):
    if(sampled=='stratified'):
        df_=df.groupby(col, group_keys=False).apply(lambda x: x.sample(int(np.rint(n_samples*len(x)/len(df))))).sample(frac=1,random_state=random_state).reset_index(drop=True)
    
    elif(sampled=='equal'):
        df_=df.groupby(col, group_keys=False).apply(lambda x: x.sample(int(n_samples/2))).sample(frac=1,random_state=random_state).reset_index(drop=True)
    
    return df_

In [ ]:
def return_cnngru_dataloader(tuples,batch_size=8,is_train=False):
    input_ids=[ele[0] for ele in tuples]
    labels=[ele[1] for ele in tuples]
    inputs = torch.tensor(input_ids)
    labels = torch.tensor(labels,dtype=torch.long)

    data = TensorDataset(inputs, labels)
    if(is_train==False):
        sampler = SequentialSampler(data)
    else:
        sampler = RandomSampler(data)
 
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

    return dataloader

## Multilingual Utils

In [ ]:
# Get merged vectors, train, val and test data for all languages
def create_multilingual_data_split(multilingual_params):
    lang_to_vec = {}
    lang_to_train = {}
    lang_to_val = {}
    lang_to_test = {} 
    lang_to_word2id = {}
    for lang in multilingual_params['languages']:
        print('Splitting data for language: ', lang)
        # Load the datasets
        csv_file_path = multilingual_params['data_dir']+'/'+lang+'_final.csv'
        X, y = split_csv_into_xy(csv_file_path)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
        val_size = 0.1/0.9
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size)

        df_train, df_test, df_val = create_data_dfs(X_train, y_train, X_test, y_test, X_val, y_val, multilingual_params)
        
        path=f'{EMBEDDINGS_PATH}/wiki.multi.'+lang_map[lang]+'.vec'
        if lang == 'hindi':
            path = f'{EMBEDDINGS_PATH}/vectors-hi.txt'

        vector,id2word,word2id=load_vec(path)
        train_data=encode_data(df_train,word2id)
        val_data=encode_data(df_val,word2id)
        test_data=encode_data(df_test,word2id)

        pad_vec=np.random.randn(1,300) 
        unk_vec=np.random.randn(1,300)
        merged_vec=np.append(vector, unk_vec, axis=0)
        merged_vec=np.append(merged_vec, pad_vec, axis=0)

        # Create lang data dictionaries
        lang_to_vec[lang] = merged_vec
        lang_to_train[lang] = train_data
        lang_to_val[lang] = val_data
        lang_to_test[lang] = test_data
        lang_to_word2id[lang] = word2id
        
    return lang_to_vec, lang_to_train, lang_to_val, lang_to_test, lang_to_word2id

In [ ]:
def return_multi_cnngru_test_dataloader(tuples, lang, word2id, batch_size=8,is_train=False):

    data = MonolingualDataset(tuples, lang, word2id)

    if(is_train==False):
        sampler = SequentialSampler(data)
    else:
        sampler = RandomSampler(data)
 
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

    return dataloader

In [ ]:
# languages = ['arabic', 'danish', 'english', 'french', 'german', 'hindi', 'indonesian', 'italian', 'portuguese', 'spanish', 'turkish']
# multilingual_params['languages'] = languages
# lang_to_vec, lang_to_train, lang_to_val, lang_to_test, lang_to_word2id = create_multilingual_data_split(multilingual_params)

In [ ]:
def return_multilingual_cnngru_dataloader(lang_to_data, lang_to_word2id, batch_size=8,is_train=False):
    data = MultilingualHatespeechDataset(lang_to_data, lang_to_word2id)
    if(is_train==False):
        sampler = SequentialSampler(data)
    else:
        sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

    return dataloader

## Monolingual TRAINING LOOP

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
# Main training function
def cnn_gru_train_model(params):
    # Load the datasets
    X, y = split_csv_into_xy(params['csv_file'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    val_size = 0.1/0.9
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size)

    df_train, df_test, df_val = create_data_dfs(X_train, y_train, X_test, y_test, X_val, y_val, params)
    
    path=f'{EMBEDDINGS_PATH}/wiki.multi.'+lang_map[params['language']]+'.vec'
    if params['language'] == 'hindi':
        path = f'{EMBEDDINGS_PATH}/vectors-hi.txt'

    vector,id2word,word2id=load_vec(path)
    train_data=encode_data(df_train,word2id)
    val_data=encode_data(df_val,word2id)
    test_data=encode_data(df_test,word2id)
    
    pad_vec=np.random.randn(1,300) 
    unk_vec=np.random.randn(1,300)
    merged_vec=np.append(vector, unk_vec, axis=0)
    merged_vec=np.append(merged_vec, pad_vec, axis=0)
    params['vocab_size']=merged_vec.shape[0]    

    # Generate the dataloaders
    train_dataloader = return_cnngru_dataloader(train_data,batch_size=params['batch_size'],is_train=params['is_train'])
    validation_dataloader=return_cnngru_dataloader(val_data,batch_size=params['batch_size'],is_train=False)
    test_dataloader = return_cnngru_dataloader(test_data, batch_size=params['batch_size'], is_train=False)
    
    model=CNN_GRU(params,merged_vec)
    # Tell pytorch to run this model on the GPU.
    model = model.to(device)


    optimizer = AdamW(model.parameters(),
                  lr = params['learning_rate'], # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = params['epsilon'] # args.adam_epsilon  - default is 1e-8.
                )

    fix_the_random(seed_val = params['random_seed'])
    # Store the average loss after each epoch so we can plot them.
    loss_values = []

    # bert_model = params['path_files']
    # language  = params['language']
    # name_one=bert_model+"_"+language
    # if(params['take_target']):
    #     name_one += '_target'
    
    # For each epo=ch...
    best_val_fscore=0
    best_test_fscore=0
    best_val_auc=0
    best_test_auc=0

    for epoch_i in range(0, params['epochs']):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, params['epochs']))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_loss = 0
        model.train()

        # For each batch of training data...
        for step, batch in tqdm(enumerate(train_dataloader)):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_labels = batch[1].to(device)
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            outputs = model(b_input_ids,b_labels)
            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple.
            loss = outputs[0]

           
            if step % 40 == 0 and not step == 0:
                print('batch_loss',loss)

            #Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()
            # Update the learning rate.
            #scheduler.step()
            
        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)
        train_fscore,train_accuracy,train_auc=Eval_phase_cnngru(params,'train',model,train_data,train_dataloader)
        
        print('avg_train_loss',avg_train_loss)
        print('train_fscore',train_fscore)
        print('train_accuracy',train_accuracy)

        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)
        val_fscore,val_accuracy,val_auc=Eval_phase_cnngru(params,'val',model,val_data, validation_dataloader)  
        test_fscore,test_accuracy,test_auc=Eval_phase_cnngru(params,'test',model,test_data, test_dataloader)

        #Report the final accuracy for this validation run.
    
        if(val_fscore > best_val_fscore):
            print(val_fscore,best_val_fscore)
            best_val_fscore=val_fscore
            best_test_fscore=test_fscore
            #save_model(model,tokenizer,params)
    
        if(val_auc > best_val_auc):
            print(val_auc,best_val_auc)
            best_val_auc=val_auc
            best_test_auc=test_auc
    print('best_test_fscore',best_test_fscore)
    print('best_val_fscore',best_val_fscore)
    print('best_test_auc',best_test_auc)
    print('best_val_auc',best_val_auc)
    # del model
    torch.cuda.empty_cache()
    return model                           
                                                
                                    

In [ ]:

params={
    'language':'german',
    'is_train':True,
    'is_model':True,
    'csv_file':'/content/drive/MyDrive/College/DSN Project/CNN-GRU/data/german_final.csv',
    'learning_rate':1e-4,
    'files':'../Dataset',
    'samp_strategy':'stratified',
    'epsilon':1e-8,
    'path_files':'cnngru',
    'take_ratio':True,
    'sample_ratio':100,
    'how_train':'baseline',
    'epochs':20,
    'batch_size':16,
    'to_save':True,
    'weights':[1.0,1.0],
    'what_bert':'normal',
    'save_only_bert':False,
    'max_length':128,
    'columns_to_consider':['directness','target','group'],
    'random_seed':42,
    'embed_size':300,
    'train_embed':True,
    'take_target':False,
    'pair':False
}

## Multilingual TRAINING LOOP

In [ ]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cpu')
print(device)

cpu


In [ ]:
# Main training function
def cnn_gru_multilingual_train_model(m_params):
    
    # Get lang mapped to data
    lang_to_vec, lang_to_train, lang_to_val, lang_to_test, lang_to_word2id = create_multilingual_data_split(m_params)

    # Generate the dataloaders
    train_dataloader = return_multilingual_cnngru_dataloader(lang_to_train, lang_to_word2id, batch_size=m_params['batch_size'], is_train=m_params['is_train'])
    # train_dataloaders = {}
    # for lang in lang_to_train:
    #     train_data = lang_to_test[lang]
    #     train_dataloader = return_cnngru_dataloader(train_data, batch_size=m_params['batch_size'], is_train=True)
    #     train_dataloaders[lang] = train_dataloader
    validation_dataloader = return_multilingual_cnngru_dataloader(lang_to_val, lang_to_word2id, batch_size=m_params['batch_size'], is_train=False)
    # for lang in lang_to_val:
    #     val_data = lang_to_val[lang]
    #     val_dataloader = return_cnngru_dataloader(val_data, batch_size=m_params['batch_size'], is_train=True)
    #     val_dataloaders[lang] = val_dataloader
    test_dataloaders = {}
    for lang in lang_to_test:
        test_data = lang_to_test[lang]
        test_dataloader = return_multi_cnngru_test_dataloader(test_data, lang, lang_to_word2id[lang], batch_size=m_params['batch_size'], is_train=False)
        test_dataloaders[lang] = test_dataloader
    
    model = MULTI_CNN_GRU(m_params,lang_to_vec)
    # Tell pytorch to run this model on the GPU.
    model = model.to(device)


    optimizer = AdamW(model.parameters(),
                  lr = m_params['learning_rate'], # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = m_params['epsilon'] # args.adam_epsilon  - default is 1e-8.
                )

    fix_the_random(seed_val = m_params['random_seed'])
    # Store the average loss after each epoch so we can plot them.
    loss_values = []


    
    # For each epo=ch...
    best_val_fscore=0
    best_test_fscore=0
    best_val_auc=0
    best_test_auc=0

    for epoch_i in range(0, m_params['epochs']):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, m_params['epochs']))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_loss = 0
        model.train()

        # For each batch of training data...
        for step, batch in tqdm(enumerate(train_dataloader)):
            
            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: labels 
            #   [2]: langs
            b_input_ids = batch[0].to(device)
            b_labels = batch[1].to(device)
            b_langs = torch.tensor([LANG_CODE_MAP[lang] for lang in batch[2]]).to(device)
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            outputs = model(b_input_ids,b_labels,b_langs)
            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple.
            loss = outputs[0]

           
            if step % 40 == 0 and not step == 0:
                print('batch_loss',loss)

            #Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()
            # Update the learning rate.
            #scheduler.step()
            
        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)
        train_fscore,train_accuracy,train_auc=Eval_phase_multi_cnngru(m_params,'train',model,train_dataloader)
        
        print('avg_train_loss',avg_train_loss)
        print('train_fscore',train_fscore)
        print('train_accuracy',train_accuracy)

        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)
        val_fscore,val_accuracy,val_auc=Eval_phase_multi_cnngru(m_params,'val',model, validation_dataloader) 

        lang_to_test_scores = defaultdict(lambda:defaultdict(float))
        for lang in test_dataloaders:
            test_loader = test_dataloaders[lang]
            test_fscore,test_accuracy,test_auc=Eval_phase_multi_cnngru(m_params,'test',model, test_loader)
            if test_fscore > lang_to_test_scores[lang]['fscore']:
                lang_to_test_scores[lang]['fscore'] = test_fscore
            if test_accuracy > lang_to_test_scores[lang]['accuracy']:
                lang_to_test_scores[lang]['accuracy'] = test_accuracy
            if test_auc > lang_to_test_scores[lang]['auc']:
                lang_to_test_scores[lang]['auc'] = test_auc

        #Report the final accuracy for this validation run.
    
        if(val_fscore > best_val_fscore):
            print(val_fscore,best_val_fscore)
            best_val_fscore=val_fscore                
            #save_model(model,tokenizer,params)
    
        if(val_auc > best_val_auc):
            print(val_auc,best_val_auc)
            best_val_auc=val_auc
    
    print('------F1-SCORE------')
    for lang in lang_to_test_scores:
        print(f'{lang} best_test_fscore: ', lang_to_test_scores[lang]['fscore'])
        print(f'{lang} best_test_auc: ', lang_to_test_scores[lang]['auc'])

    # del model
    torch.cuda.empty_cache()
    return model                           
                                                
                                    

In [ ]:
multilingual_params = {
    'languages': ['arabic', 'danish', 'english', 'french', 'german', 'hindi', 'indonesian', 'italian', 'portuguese', 'spanish', 'turkish'],
    'germanic_languages': ['english', 'german', 'danish'],
    'romance_languages': ['spanish', 'italian', 'french', 'portuguese'],
    'data_dir': '/content/drive/MyDrive/DSN Project/CNN-GRU/data',
    'is_multilingual_test':False,
    'is_train':True,
    'is_model':True,
    'learning_rate':1e-4,
    'files':'../Dataset',
    'samp_strategy':'stratified',
    'epsilon':1e-8,
    'path_files':'cnngru',
    'take_ratio':True,
    'sample_ratio':100,
    'how_train':'baseline',
    'epochs':20,
    'batch_size':16,
    'to_save':True,
    'weights':[1.0,1.0],
    'what_bert':'normal',
    'save_only_bert':False,
    'max_length':128,
    'columns_to_consider':['directness','target','group'],
    'random_seed':42,
    'embed_size':300,
    'train_embed':True,
    'take_target':False,
    'pair':False

}

# Monolingual Train Monolingual Test Runs

### Monolingual German

In [ ]:
model = cnn_gru_train_model(params)

100%|██████████| 697/697 [00:00<00:00, 13080.11it/s]



======== Epoch 1 / 20 ========
Training...


48it [00:00, 76.60it/s]

batch_loss tensor(0.3881, device='cuda:0', grad_fn=<NllLossBackward0>)


96it [00:01, 77.36it/s]

...

batch_loss tensor(0.0349, device='cuda:0', grad_fn=<NllLossBackward0>)


328it [00:04, 75.22it/s]

batch_loss tensor(0.0772, device='cuda:0', grad_fn=<NllLossBackward0>)


348it [00:04, 76.33it/s]


Running eval on  train ...
 Accuracy: 0.9750359195402298
 Fscore: 0.97
 Roc Auc score: 0.95
 Test took: 0:00:01
avg_train_loss 0.06093821698972345
train_fscore 0.9746009068360968
train_accuracy 0.9750359195402298
Running eval on  val ...
 Accuracy: 0.8952654232424677
 Fscore: 0.89
 Roc Auc score: 0.78
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.8880918220946915
 Fscore: 0.88
 Roc Auc score: 0.79
 Test took: 0:00:00
0.8891160018675447 0.8879585173804495
best_test_fscore 0.8806200230617137
best_val_fscore 0.8891160018675447
best_test_auc 0.8050470182121177
best_val_auc 0.8013591800356505


### Monolingual Danish

In [ ]:
params['language'] = 'danish'
params['csv_file'] = f'{CSV_FILE_PATH}/danish_final.csv'
model = cnn_gru_train_model(params)

100%|██████████| 328/328 [00:00<00:00, 12273.46it/s]



======== Epoch 1 / 20 ========
Training...


48it [00:00, 57.26it/s]

batch_loss tensor(0.6976, device='cuda:0', grad_fn=<NllLossBackward0>)


90it [00:01, 57.70it/s]

...

batch_loss tensor(0.0378, device='cuda:0', grad_fn=<NllLossBackward0>)


126it [00:02, 57.15it/s]

batch_loss tensor(0.2314, device='cuda:0', grad_fn=<NllLossBackward0>)


164it [00:02, 57.24it/s]


batch_loss tensor(0.2374, device='cuda:0', grad_fn=<NllLossBackward0>)
Running eval on  train ...
 Accuracy: 0.9629629629629629
 Fscore: 0.96
 Roc Auc score: 0.89
 Test took: 0:00:01
avg_train_loss 0.16479656501196144
train_fscore 0.9615562610443598
train_accuracy 0.9629629629629629
Running eval on  val ...
 Accuracy: 0.8810975609756098
 Fscore: 0.87
 Roc Auc score: 0.61
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.8689024390243902
 Fscore: 0.85
 Roc Auc score: 0.59
 Test took: 0:00:00
0.868828771101172 0.8641521675803079
0.6063871282301316 0.5520234032179424
best_test_fscore 0.852507568401211
best_val_fscore 0.868828771101172
best_test_auc 0.5928932659036465
best_val_auc 0.6063871282301316


### Monolingual English

In [ ]:
params['language'] = 'english'
params['csv_file'] = f'{CSV_FILE_PATH}/english_final.csv'
model = cnn_gru_train_model(params)

100%|██████████| 8195/8195 [00:00<00:00, 14240.53it/s]



======== Epoch 1 / 20 ========
Training...


57it [00:00, 91.98it/s]

batch_loss tensor(0.4844, device='cuda:0', grad_fn=<NllLossBackward0>)


97it [00:01, 95.50it/s]

...

4051it [00:45, 90.26it/s]

batch_loss tensor(0.0090, device='cuda:0', grad_fn=<NllLossBackward0>)


4098it [00:45, 89.91it/s]

batch_loss tensor(0.2314, device='cuda:0', grad_fn=<NllLossBackward0>)
Running eval on  train ...


 Accuracy: 0.9877658113921347
 Fscore: 0.99
 Roc Auc score: 0.99
 Test took: 0:00:11
avg_train_loss 0.039842761173888235
train_fscore 0.9878488835892675
train_accuracy 0.9877658113921347
Running eval on  val ...
 Accuracy: 0.7928972418843055
 Fscore: 0.80
 Roc Auc score: 0.69
 Test took: 0:00:01
Running eval on  test ...
 Accuracy: 0.787797437461867
 Fscore: 0.80
 Roc Auc score: 0.68
 Test took: 0:00:01
best_test_fscore 0.8512513302900171
best_val_fscore 0.860174895675484
best_test_auc 0.7268730041758781
best_val_auc 0.7400200308256274


### Monolingual French

In [ ]:
params['language'] = 'french'
params['csv_file'] = f'{CSV_FILE_PATH}/french_final.csv'
model = cnn_gru_train_model(params)

100%|██████████| 103/103 [00:00<00:00, 12755.04it/s]



======== Epoch 1 / 20 ========
Training...


52it [00:00, 150.55it/s]


batch_loss tensor(0.3859, device='cuda:0', grad_fn=<NllLossBackward0>)
Running eval on  train ...
 Accuracy: 0.8017031630170316
 Fscore: 0.71
 Roc Auc score: 0.50
 Test took: 0:00:00
avg_train_loss 0.5120533412465682
train_fscore 0.713467095784232
train_accuracy 0.8017031630170316
Running eval on  val ...
 Accuracy: 0.8155339805825242
 Fscore: 0.73
 Roc Auc score: 0.50
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.7572815533980582
 Fscore: 0.65
 Roc Auc score: 0.50
 Test took: 0:00:00
0.7326722392399148 0
0.5 0

======== Epoch 2 / 20 ========
Training...


52it [00:00, 155.51it/s]


...

batch_loss tensor(0.5689, device='cuda:0', grad_fn=<NllLossBackward0>)
Running eval on  train ...
 Accuracy: 0.8272506082725061
 Fscore: 0.77
 Roc Auc score: 0.57
 Test took: 0:00:00
avg_train_loss 0.42133713456300587
train_fscore 0.7749394869397358
train_accuracy 0.8272506082725061
Running eval on  val ...
 Accuracy: 0.8155339805825242
 Fscore: 0.73
 Roc Auc score: 0.50
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.7572815533980582
 Fscore: 0.65
 Roc Auc score: 0.50
 Test took: 0:00:00

======== Epoch 19 / 20 ========
Training...


52it [00:00, 155.64it/s]


batch_loss tensor(0.4077, device='cuda:0', grad_fn=<NllLossBackward0>)
Running eval on  train ...
 Accuracy: 0.816301703163017
 Fscore: 0.75
 Roc Auc score: 0.54
 Test took: 0:00:00
avg_train_loss 0.3909802812223251
train_fscore 0.7483112818122523
train_accuracy 0.816301703163017
Running eval on  val ...
 Accuracy: 0.8155339805825242
 Fscore: 0.73
 Roc Auc score: 0.50
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.7572815533980582
 Fscore: 0.65
 Roc Auc score: 0.50
 Test took: 0:00:00

======== Epoch 20 / 20 ========
Training...


52it [00:00, 152.24it/s]


batch_loss tensor(0.3029, device='cuda:0', grad_fn=<NllLossBackward0>)
Running eval on  train ...
 Accuracy: 0.8454987834549879
 Fscore: 0.81
 Roc Auc score: 0.62
 Test took: 0:00:00
avg_train_loss 0.3691670728417543
train_fscore 0.8074862454822027
train_accuracy 0.8454987834549879
Running eval on  val ...
 Accuracy: 0.8155339805825242
 Fscore: 0.73
 Roc Auc score: 0.50
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.7572815533980582
 Fscore: 0.65
 Roc Auc score: 0.50
 Test took: 0:00:00
best_test_fscore 0.6526846537574424
best_val_fscore 0.7326722392399148
best_test_auc 0.5
best_val_auc 0.5


### Monolingual Arabic

In [ ]:
params['language'] = 'arabic'
params['csv_file'] = f'{CSV_FILE_PATH}/arabic_final.csv'
model = cnn_gru_train_model(params)

100%|██████████| 412/412 [00:00<00:00, 10962.93it/s]



======== Epoch 1 / 20 ========
Training...


60it [00:00, 144.04it/s]

batch_loss tensor(0.3959, device='cuda:0', grad_fn=<NllLossBackward0>)


105it [00:00, 141.44it/s]

...

batch_loss tensor(0.1398, device='cuda:0', grad_fn=<NllLossBackward0>)


180it [00:01, 141.36it/s]

batch_loss tensor(0.5246, device='cuda:0', grad_fn=<NllLossBackward0>)


206it [00:01, 141.08it/s]


batch_loss tensor(0.4118, device='cuda:0', grad_fn=<NllLossBackward0>)
Running eval on  train ...
 Accuracy: 0.8891588217430914
 Fscore: 0.84
 Roc Auc score: 0.51
 Test took: 0:00:00
avg_train_loss 0.3091349674443018
train_fscore 0.8393190148611049
train_accuracy 0.8891588217430914
Running eval on  val ...
 Accuracy: 0.8713592233009708
 Fscore: 0.82
 Roc Auc score: 0.50
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.8907766990291263
 Fscore: 0.85
 Roc Auc score: 0.52
 Test took: 0:00:00
best_test_fscore 0.8444855874162716
best_val_fscore 0.8216254080331752
best_test_auc 0.5190068899976241
best_val_auc 0.5070338384661344


### Monolingual Indonesian

In [ ]:
params['language'] = 'indonesian'
params['csv_file'] = f'{CSV_FILE_PATH}/indonesian_final.csv'
model = cnn_gru_train_model(params)

100%|██████████| 1389/1389 [00:00<00:00, 15048.45it/s]



======== Epoch 1 / 20 ========
Training...


60it [00:00, 138.90it/s]

batch_loss tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)


103it [00:00, 138.80it/s]

...

batch_loss tensor(0.0032, device='cuda:0', grad_fn=<NllLossBackward0>)


660it [00:04, 133.56it/s]

batch_loss tensor(0.6300, device='cuda:0', grad_fn=<NllLossBackward0>)


694it [00:05, 136.16it/s]


batch_loss tensor(0.0026, device='cuda:0', grad_fn=<NllLossBackward0>)
Running eval on  train ...
 Accuracy: 0.9773054755043228
 Fscore: 0.98
 Roc Auc score: 0.98
 Test took: 0:00:01
avg_train_loss 0.08561777316497823
train_fscore 0.9773170034093488
train_accuracy 0.9773054755043228
Running eval on  val ...
 Accuracy: 0.7789776817854571
 Fscore: 0.77
 Roc Auc score: 0.76
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.7494600431965442
 Fscore: 0.75
 Roc Auc score: 0.74
 Test took: 0:00:00
best_test_fscore 0.7747147645406911
best_val_fscore 0.7914834724546224
best_test_auc 0.7724329413164797
best_val_auc 0.7786533678493477


### Monolingual Italian

In [ ]:
params['language'] = 'italian'
params['csv_file'] = f'{CSV_FILE_PATH}/italian_final.csv'
model = cnn_gru_train_model(params)

100%|██████████| 1212/1212 [00:00<00:00, 14373.64it/s]



======== Epoch 1 / 20 ========
Training...


58it [00:00, 139.99it/s]

batch_loss tensor(0.5606, device='cuda:0', grad_fn=<NllLossBackward0>)


101it [00:00, 135.61it/s]

...

batch_loss tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)


576it [00:04, 137.92it/s]

batch_loss tensor(0.0010, device='cuda:0', grad_fn=<NllLossBackward0>)


606it [00:04, 139.06it/s]


batch_loss tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
Running eval on  train ...
 Accuracy: 0.9979364424267437
 Fscore: 1.00
 Roc Auc score: 1.00
 Test took: 0:00:01
avg_train_loss 0.01697892752489868
train_fscore 0.9979343789056474
train_accuracy 0.9979364424267437
Running eval on  val ...
 Accuracy: 0.8415841584158416
 Fscore: 0.84
 Roc Auc score: 0.78
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.8135313531353136
 Fscore: 0.81
 Roc Auc score: 0.76
 Test took: 0:00:00
best_test_fscore 0.8046859621434764
best_val_fscore 0.8456389132455114
best_test_auc 0.7930109819305078
best_val_auc 0.7953096546911963


### Monolingual Portuguese

In [ ]:
params['language'] = 'portuguese'
params['csv_file'] = f'{CSV_FILE_PATH}/portuguese_final.csv'
model = cnn_gru_train_model(params)

100%|██████████| 567/567 [00:00<00:00, 13346.19it/s]



======== Epoch 1 / 20 ========
Training...


63it [00:00, 154.80it/s]

batch_loss tensor(0.4527, device='cuda:0', grad_fn=<NllLossBackward0>)


95it [00:00, 152.59it/s]

...

batch_loss tensor(0.0152, device='cuda:0', grad_fn=<NllLossBackward0>)


266it [00:01, 151.43it/s]

batch_loss tensor(0.0055, device='cuda:0', grad_fn=<NllLossBackward0>)


284it [00:01, 148.11it/s]


batch_loss tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
Running eval on  train ...
 Accuracy: 0.9880899867666519
 Fscore: 0.99
 Roc Auc score: 0.97
 Test took: 0:00:00
avg_train_loss 0.028741857905400355
train_fscore 0.987994856588111
train_accuracy 0.9880899867666519
Running eval on  val ...
 Accuracy: 0.8218694885361552
 Fscore: 0.79
 Roc Auc score: 0.63
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.8518518518518519
 Fscore: 0.83
 Roc Auc score: 0.67
 Test took: 0:00:00
best_test_fscore 0.8377791088482912
best_val_fscore 0.8170582623953563
best_test_auc 0.7170940170940171
best_val_auc 0.6953382181161729


### Monolingual Spanish

In [ ]:
params['language'] = 'spanish'
params['csv_file'] = f'{CSV_FILE_PATH}/spanish_final.csv'
model = cnn_gru_train_model(params)

100%|██████████| 1260/1260 [00:00<00:00, 13727.99it/s]



======== Epoch 1 / 20 ========
Training...


60it [00:00, 144.16it/s]

batch_loss tensor(0.6729, device='cuda:0', grad_fn=<NllLossBackward0>)


105it [00:00, 137.89it/s]

...

batch_loss tensor(0.0039, device='cuda:0', grad_fn=<NllLossBackward0>)


622it [00:04, 141.28it/s]

batch_loss tensor(0.2801, device='cuda:0', grad_fn=<NllLossBackward0>)


630it [00:04, 136.89it/s]


Running eval on  train ...
 Accuracy: 0.9763888888888889
 Fscore: 0.98
 Roc Auc score: 0.97
 Test took: 0:00:01
avg_train_loss 0.0978366670568311
train_fscore 0.9764184772028781
train_accuracy 0.9763888888888889
Running eval on  val ...
 Accuracy: 0.6793650793650794
 Fscore: 0.68
 Roc Auc score: 0.66
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.6777777777777778
 Fscore: 0.68
 Roc Auc score: 0.65
 Test took: 0:00:00
best_test_fscore 0.724364672997631
best_val_fscore 0.736669242361416
best_test_auc 0.6809050154104791
best_val_auc 0.6954859466070228


### Monolingual Turkish

In [ ]:
params['language'] = 'turkish'
params['csv_file'] = f'{CSV_FILE_PATH}/turkish_final.csv'
model = cnn_gru_train_model(params)

100%|██████████| 3480/3480 [00:00<00:00, 13810.43it/s]



======== Epoch 1 / 20 ========
Training...


44it [00:02, 19.21it/s]

batch_loss tensor(0.5570, device='cuda:0', grad_fn=<NllLossBackward0>)


84it [00:04, 19.46it/s]

...

batch_loss tensor(0.0272, device='cuda:0', grad_fn=<NllLossBackward0>)


1724it [01:29, 19.39it/s]

batch_loss tensor(0.2883, device='cuda:0', grad_fn=<NllLossBackward0>)


1740it [01:30, 19.30it/s]


Running eval on  train ...
 Accuracy: 0.9697111238861742
 Fscore: 0.97
 Roc Auc score: 0.94
 Test took: 0:00:33
avg_train_loss 0.12346597070278514
train_fscore 0.9694521645599614
train_accuracy 0.9697111238861742
Running eval on  val ...
 Accuracy: 0.7660919540229885
 Fscore: 0.75
 Roc Auc score: 0.59
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.7635057471264368
 Fscore: 0.76
 Roc Auc score: 0.61
 Test took: 0:00:01
best_test_fscore 0.7902449232662015
best_val_fscore 0.794633963759655
best_test_auc 0.6580934604233386
best_val_auc 0.6580473383798364


### Monolingual Hindi

In [ ]:
params['language'] = 'hindi'
params['csv_file'] = f'{CSV_FILE_PATH}/hindi_final.csv'
model = cnn_gru_train_model(params)

100%|██████████| 1500/1500 [00:00<00:00, 8663.36it/s]



======== Epoch 1 / 20 ========
Training...


44it [00:01, 44.91it/s]

batch_loss tensor(0.5005, device='cuda:0', grad_fn=<NllLossBackward0>)


85it [00:02, 49.57it/s]

...

batch_loss tensor(0.2505, device='cuda:0', grad_fn=<NllLossBackward0>)


728it [00:14, 49.02it/s]

batch_loss tensor(0.3046, device='cuda:0', grad_fn=<NllLossBackward0>)


750it [00:15, 49.48it/s]


Running eval on  train ...
 Accuracy: 0.91475
 Fscore: 0.91
 Roc Auc score: 0.79
 Test took: 0:00:05
avg_train_loss 0.23434833535552024
train_fscore 0.9071755600382618
train_accuracy 0.91475
Running eval on  val ...
 Accuracy: 0.8213333333333334
 Fscore: 0.81
 Roc Auc score: 0.64
 Test took: 0:00:00
Running eval on  test ...
 Accuracy: 0.824
 Fscore: 0.80
 Roc Auc score: 0.64
 Test took: 0:00:00
best_test_fscore 0.8066422318616407
best_val_fscore 0.8261716422234153
best_test_auc 0.7008333333333333
best_val_auc 0.6930495505621257


# Multilingual Train Monolingual Test

In [ ]:
# ['arabic', 'danish', 'english', 'french', 'german', 'hindi', 'indonesian', 'italian', 'portuguese', 'spanish', 'turkish'],
multilingual_params['languages'] = multilingual_params['languages']
cnn_gru_multilingual_train_model(multilingual_params)

Splitting data for language:  arabic


100%|██████████| 412/412 [00:00<00:00, 14421.23it/s]


Splitting data for language:  danish


100%|██████████| 328/328 [00:00<00:00, 15117.60it/s]


Splitting data for language:  english


...

batch_loss tensor(0.3115, grad_fn=<NllLossBackward0>)


9560it [1:23:31,  1.88it/s]

batch_loss tensor(0.5960, grad_fn=<NllLossBackward0>)


9569it [1:23:36,  1.91it/s]


Running eval on  train ...


In [ ]:
# multilingual_params['languages'] = multilingual_params['romance_languages']
multilingual_params['languages'] = multilingual_params['germanic_languages']
cnn_gru_multilingual_train_model(multilingual_params)

In [ ]:
params['language'] = 'french'
params['csv_file'] = f'{CSV_FILE_PATH}/french_final.csv'
model = cnn_gru_train_model(params)

100%|██████████| 103/103 [00:00<00:00, 14420.63it/s]



======== Epoch 1 / 20 ========
Training...


0it [00:00, ?it/s]

torch.Size([16, 26, 300])


16it [00:00, 57.97it/s]

...

torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])


42it [00:00, 134.25it/s]

torch.Size([16, 26, 300])
batch_loss tensor(0.5367, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])
torch.Size([16, 26, 300])


44it [00:00, 132.55it/s]


KeyboardInterrupt: ignored

In [ ]:
# prints currently alive Tensors and Variables
import torch
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except:
        pass

<class 'torch.nn.parameter.Parameter'> torch.Size([50002, 300])
<class 'torch.nn.parameter.Parameter'> torch.Size([50002, 300])
<class 'torch.nn.parameter.Parameter'> torch.Size([100, 300, 2])
<class 'torch.nn.parameter.Parameter'> torch.Size([100])
<class 'torch.nn.parameter.Parameter'> torch.Size([100, 300, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([100])
<class 'torch.nn.parameter.Parameter'> torch.Size([100, 300, 4])
<class 'torch.nn.parameter.Parameter'> torch.Size([100])
<class 'torch.nn.parameter.Parameter'> torch.Size([300, 100])
<class 'torch.nn.parameter.Parameter'> torch.Size([300, 100])
<class 'torch.nn.parameter.Parameter'> torch.Size([300])
<class 'torch.nn.parameter.Parameter'> torch.Size([300])
<class 'torch.nn.parameter.Parameter'> torch.Size([2, 100])
<class 'torch.nn.parameter.Parameter'> torch.Size([2])
<class 'torch.Tensor'> torch.Size([1, 427])
<class 'torch.Tensor'> torch.Size([1, 1])
<class 'torch.Tensor'> torch.Size([1, 427])
<class 'torch.Tensor'> 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: astor.all_symbols is deprecated.  Please use astor.symbol_data.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: astor.treewalk is deprecated.  Please use astor.tree_walk.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: astor.codegen is deprecated.  Please use astor.code_gen.
  
/usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py:171: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  "torch.distributed.reduce_op is deprecated, please use "


<class 'torch.Tensor'> torch.Size([1, 427])
<class 'torch.Tensor'> torch.Size([1, 1])
<class 'torch.Tensor'> torch.Size([1, 1])
<class 'torch.Tensor'> torch.Size([1, 427])
<class 'torch.Tensor'> torch.Size([697])
<class 'torch.Tensor'> torch.Size([697, 244])
<class 'torch.nn.parameter.Parameter'> torch.Size([2])
<class 'torch.nn.parameter.Parameter'> torch.Size([2, 100])
<class 'torch.nn.parameter.Parameter'> torch.Size([300])
<class 'torch.nn.parameter.Parameter'> torch.Size([300])
<class 'torch.nn.parameter.Parameter'> torch.Size([300, 100])
<class 'torch.nn.parameter.Parameter'> torch.Size([300, 100])
<class 'torch.nn.parameter.Parameter'> torch.Size([100])
<class 'torch.nn.parameter.Parameter'> torch.Size([100, 300, 4])
<class 'torch.nn.parameter.Parameter'> torch.Size([100])
<class 'torch.nn.parameter.Parameter'> torch.Size([100, 300, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([100])
<class 'torch.nn.parameter.Parameter'> torch.Size([100, 300, 2])
<class 'torch.Tensor'>